In [1]:
gpu_info_examine = !nvidia-smi
gpu_info_examine = '\n'.join(gpu_info_examine)
if gpu_info_examine.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info_examine)

Tue Jun 29 05:54:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    41W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install tqdm
!pip install numpy

In [4]:
from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, BartTokenizerFast
from datasets import load_dataset
import torch as torch
from tqdm import tqdm
from datasets import load_metric
import numpy as np

In [5]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [6]:
model_checkpoint = 'facebook/bart-large-cnn' #'facebook/bart-base' #'a1noack/bart-large-gigaword'
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [8]:
test = load_dataset("cnn_dailymail", '3.0.0',split='test[15:20]')
# test = load_dataset("cnn_dailymail", '3.0.0',split='test[10:15]')
# test = load_dataset("cnn_dailymail", '3.0.0',split='test[5:10]')
# test = load_dataset("cnn_dailymail", '3.0.0',split='test[0:5]')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


In [9]:
print(test)
print(model.config.max_length)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 5
})
142


In [10]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [11]:
encodings =  tokenizer(test['article'], return_tensors='pt', padding=True, truncation=True, max_length=1024).to(device)

In [12]:
print(encodings['input_ids'].shape)
print(encodings['attention_mask'].shape)
encodings

torch.Size([5, 1024])
torch.Size([5, 1024])


{'input_ids': tensor([[    0,   530,  2681,  ...,    11,    63,     2],
        [    0,  1640, 16256,  ...,     1,     1,     1],
        [    0,  1640, 16256,  ...,  9095,     6,     2],
        [    0, 32826,    36,  ...,     1,     1,     1],
        [    0, 33193, 19927,  ...,  1699,     6,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [13]:
# encodings['input_ids'][:5].shape
# encodings['input_ids'][10:].shape

In [14]:
model = model.to(device)
model.eval()
number_beams = 8

In [15]:
with torch.no_grad():
      result1 = model.generate(encodings['input_ids'],  num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)
    # result = model.generate(encodings['input_ids'],  num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [16]:
all = []
log_sent = []
print(result1.sequences.shape)
print("CNN Dailymail vocab size: ", result1.scores[0].shape[1])
print("Input ids size", encodings['input_ids'].shape)
for batch_num in range(0, result1.scores[0].shape[0], number_beams):
    # lls = torch.tensor(0, dtype=torch.float)
    # print(batch_num)
    max_score = torch.tensor(-1*1e6, dtype=torch.float).to(device)
    for beam_num in range(number_beams):
        print([torch.max(result1.scores[-1][batch_num+beam_num]), max_score])
        max_score = torch.max(torch.stack([torch.max(result1.scores[-1][batch_num+beam_num]), max_score]))
    log_sent.append(max_score)

torch.Size([5, 65])
CNN Dailymail vocab size:  50264
Input ids size torch.Size([5, 1024])
[tensor(-0.8043, device='cuda:0'), tensor(-1000000., device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-0.8043, device='cuda:0'), tensor(-0.8043, device='cuda:0')]
[tensor(-34.9259, device='cuda:0'), tensor(-1000000., device='cuda:0')]
[tensor(-36.4917, device='cuda:0'), tensor(-34.9259, device='cuda:0')]
[tensor(-36.7551, device='cuda:0'), tensor(-34.9259, device='cuda:0')]
[tensor(-36.4029, device='cuda:0'), tensor(-34.9259, device='cuda:0')]
[tensor(-38.9479, device='cuda:0'), tensor(-34.9259, device='cuda:0')]


In [17]:
print(log_sent)
print(torch.stack(log_sent).sum())
print(torch.exp((-1*(torch.stack(log_sent).sum()))/result1.sequences.shape[1]))

[tensor(-0.8043, device='cuda:0'), tensor(-34.9259, device='cuda:0'), tensor(-0.6400, device='cuda:0'), tensor(-0.5767, device='cuda:0'), tensor(-0.5996, device='cuda:0')]
tensor(-37.5465, device='cuda:0')
tensor(1.7818, device='cuda:0')
